# Wrapping Potentials

In [1]:
"""
    TITLE   : Wrapping Potentials
    PROJECT : `discO`
"""

__author__ = "Nathaniel Starkman"
__version__ = "Feb 10, 2021"

<span style='font-size:30px;font-weight:650'>
    About
</span>

Gravitational potentials are managed by 3rd party packages, eg galpy.  
We provide thin wrappers to unify the I/O.  

Some nice features are:

- associating reference frames to potentials,
- use of astropy Quantities.
- vector fields for the acceleration / specific force.

<br><br>

- - - 

## Prepare


### Imports

In [2]:
# THIRD PARTY
import astropy.coordinates as coord
import astropy.units as u
from galpy.potential import NFWPotential
from IPython.display import display

# PROJECT-SPECIFIC
from discO.core.core import PotentialWrapper
from discO.plugin.galpy import GalpyPotentialWrapper

### Parameters

We define a potential,

In [3]:
pot = NFWPotential(1e12 * u.solMass)
pot

and a point at which we will evaluate the potential.

In [4]:
points_rep = coord.CartesianRepresentation(
    x=[0, 1, 2, 3] * u.kpc, y=[4, 5, 6, 7] * u.kpc, z=[8, 9, 10, 11] * u.kpc
)

points_c = coord.ICRS(points_rep)

<br><br>

- - - 

## Wrapping a Potential

In [5]:
wpot = PotentialWrapper(
    pot,  # the potential
)
wpot

GalpyPotentialWrapper: at <0x7fbbab73d9d0>
    potential : galpy.potential.TwoPowerSphericalPotential.NFWPotential object at 0x7fbbb00ca940
    frame     : None

The wrapped potential is accessible via

In [6]:
wpot.__wrapped__

Now we can evaluate the potential in its internal reference frame.  
All keyword arguments are passed to the wrapped potential.

In [7]:
p, v = wpot(points_rep)
display(p, v)

<CartesianRepresentation (x, y, z) in kpc
    [(0., 4.,  8.), (1., 5.,  9.), (2., 6., 10.), (3., 7., 11.)]>

<Quantity [-360877.67677075, -345045.88442112, -330002.72852092,
           -315991.6407802 ] km2 / s2>

Note that we had to pass a Representation object, not a CoordinateFrame or SkyCoord.
This is becuase the potential has not associated frame, so it does not know how
to convert CoordinateFrame or SkyCoord objects to its internal reference frame.

In most / all packages, Potentials do not have an associated reference frame. This can be good, in that an abstract potential does not really have or need a specific reference frame as there is not a set observer. The potential is just out "there".  
However, for connecting to a real observer, ie **us**, it is convenient to specify the reference frame of the potential.

``PotentialWrapper`` also allows a reference frame to be associated with the wrapped potential.

In [8]:
wpot = PotentialWrapper(pot, frame="galactocentric")
wpot

GalpyPotentialWrapper: at <0x7fbbb04b30a0>
    potential : galpy.potential.TwoPowerSphericalPotential.NFWPotential object at 0x7fbbb00ca940
    frame     : Galactocentric Frame (galcen_coord=<ICRS Coordinate: (ra, dec) in deg
        (266.4051, -28.936175)>, galcen_distance=8.122 kpc, galcen_v_sun=(12.9, 245.6, 7.78) km / s, z_sun=20.8 pc, roll=0.0 deg)

This wrapped potential now has a reference frame, giving the location of the origin and the observer.

Now when we evaluate the potential we can give a Representation, to use the internal reference frame,
or pass an object in any other reference frame.

The evaluated potential returns the points, in the potential's reference frame, and the value at those points.

In [9]:
p, v = wpot(points_c)
display(p, v)

<Galactocentric Coordinate (galcen_coord=<ICRS Coordinate: (ra, dec) in deg
    (266.4051, -28.936175)>, galcen_distance=8.122 kpc, galcen_v_sun=(12.9, 245.6, 7.78) km / s, z_sun=20.8 pc, roll=0.0 deg): (x, y, z) in kpc
    [(-15.47906526, 4.19654124, 2.89521132),
     (-16.89276836, 4.99280535, 2.28907024),
     (-18.30647146, 5.78906946, 1.68292915),
     (-19.72017456, 6.58533357, 1.07678807)]>

<Quantity [-293178.43903212, -283166.27712042, -273689.21695083,
           -264762.03262673] km2 / s2>

In [10]:
p, v = wpot(points_rep)
display(p, v)

<Galactocentric Coordinate (galcen_coord=<ICRS Coordinate: (ra, dec) in deg
    (266.4051, -28.936175)>, galcen_distance=8.122 kpc, galcen_v_sun=(12.9, 245.6, 7.78) km / s, z_sun=20.8 pc, roll=0.0 deg): (x, y, z) in kpc
    [(0., 4.,  8.), (1., 5.,  9.), (2., 6., 10.), (3., 7., 11.)]>

<Quantity [-360877.67677075, -345045.88442112, -330002.72852092,
           -315991.6407802 ] km2 / s2>

In addition to the specific potential (``__call__()`` or ``potential()``) we can also evaluate the acceleration field -- (``acceleration()`` or ``specific_force()``).

In [11]:
accl = wpot.acceleration(points_c)

display(accl)

<CylindricalVectorField (rho, phi, z) in (kpc, rad, kpc) | (vf_rho, vf_phi, vf_z) in km / s2
    [((16.03784336, 2.87684582, 2.89521132), (-2.27326506e-13, 0., -4.10378290e-14)),
     ((17.61515621, 2.85421537, 2.28907024), (-2.10278571e-13, 0., -2.73254698e-14)),
     ((19.2000058 , 2.83531256, 1.68292915), (-1.94294284e-13, 0., -1.70303862e-14)),
     ((20.79066865, 2.81929718, 1.07678807), (-1.79588000e-13, 0., -9.30120236e-15))]>

Although not yet implemented in the vector field, the reference frame information is preserved.

In [12]:
accl.frame

<Galactocentric Frame (galcen_coord=<ICRS Coordinate: (ra, dec) in deg
    (266.4051, -28.936175)>, galcen_distance=8.122 kpc, galcen_v_sun=(12.9, 245.6, 7.78) km / s, z_sun=20.8 pc, roll=0.0 deg)>

All the methods, including ``potential()``, take the representation type as an argument.

In [13]:
p, v = wpot(points_c, representation_type=coord.CylindricalRepresentation)
display(p, v)

accl = wpot.acceleration(points_c, representation_type="cartesian")
display(accl)

<Galactocentric Coordinate (galcen_coord=<ICRS Coordinate: (ra, dec) in deg
    (266.4051, -28.936175)>, galcen_distance=8.122 kpc, galcen_v_sun=(12.9, 245.6, 7.78) km / s, z_sun=20.8 pc, roll=0.0 deg): (x, y, z) in kpc
    [(-15.47906526, 4.19654124, 2.89521132),
     (-16.89276836, 4.99280535, 2.28907024),
     (-18.30647146, 5.78906946, 1.68292915),
     (-19.72017456, 6.58533357, 1.07678807)]>

<Quantity [-293178.43903212, -283166.27712042, -273689.21695083,
           -264762.03262673] km2 / s2>

<CartesianVectorField (x, y, z) in kpc | (vf_x, vf_y, vf_z) in km / s2
    [((-15.47906526, 4.19654124, 2.89521132), (2.19406172e-13, -5.94833753e-14, -4.10378290e-14)),
     ((-16.89276836, 4.99280535, 2.28907024), (2.01655163e-13, -5.96009461e-14, -2.73254698e-14)),
     ((-18.30647146, 5.78906946, 1.68292915), (1.85252171e-13, -5.85824357e-14, -1.70303862e-14)),
     ((-19.72017456, 6.58533357, 1.07678807), (1.70341165e-13, -5.68835427e-14, -9.30120236e-15))]>

<br><br>

- - - 

## As Static Methods

The above example showed how a potential can be wrapped.  
In addition, all of ``PotentialWrapper`` (and its subclassses) is accessible via static methods.

In [14]:
try:
    PotentialWrapper.potential(pot, points_rep)
except NotImplementedError:
    print("....")

....


However, ``PotentialWrapper`` is actually the base class for wrapping potentials from any package.
To use a method as a static method we must use the appropriate subclass for the potential.

There are 2 ways to get the appropriate wrapper class.

1. import : ``from discO.plugin.galpy import GalpyPotentialWrapper``
2. by ``getitem`` on ``PotentialWrapper`` : ``PotentialWrapper['package']``

In [15]:
PotentialWrapper["galpy"]

discO.plugin.galpy.GalpyPotentialWrapper

In [16]:
p, v = GalpyPotentialWrapper.potential(pot, points_rep, representation_type="spherical")
display(p, v)

<SphericalRepresentation (lon, lat, distance) in (rad, rad, kpc)
    [(1.57079633, 1.10714872,  8.94427191),
     (1.37340077, 1.05532979, 10.34408043),
     (1.24904577, 1.00685369, 11.83215957),
     (1.16590454, 0.96522779, 13.37908816)]>

<Quantity [-360877.67677075, -345045.88442112, -330002.72852092,
           -315991.6407802 ] km2 / s2>

To use a CoordinateFrame or SkyCoord, the frame of the potential must be specified.

In [17]:
p, v = GalpyPotentialWrapper.potential(pot, points_c, frame="galactocentric")
display(p, v)

<Galactocentric Coordinate (galcen_coord=<ICRS Coordinate: (ra, dec) in deg
    (266.4051, -28.936175)>, galcen_distance=8.122 kpc, galcen_v_sun=(12.9, 245.6, 7.78) km / s, z_sun=20.8 pc, roll=0.0 deg): (x, y, z) in kpc
    [(-15.47906526, 4.19654124, 2.89521132),
     (-16.89276836, 4.99280535, 2.28907024),
     (-18.30647146, 5.78906946, 1.68292915),
     (-19.72017456, 6.58533357, 1.07678807)]>

<Quantity [-293178.43903212, -283166.27712042, -273689.21695083,
           -264762.03262673] km2 / s2>

And similarly for the acceleration field.

In [18]:
accl = GalpyPotentialWrapper.specific_force(
    pot, points_c, frame="galactocentric", representation_type="cartesian"
)
accl

<CartesianVectorField (x, y, z) in kpc | (vf_x, vf_y, vf_z) in km / s2
    [((-15.47906526, 4.19654124, 2.89521132), (2.19406172e-13, -5.94833753e-14, -4.10378290e-14)),
     ((-16.89276836, 4.99280535, 2.28907024), (2.01655163e-13, -5.96009461e-14, -2.73254698e-14)),
     ((-18.30647146, 5.78906946, 1.68292915), (1.85252171e-13, -5.85824357e-14, -1.70303862e-14)),
     ((-19.72017456, 6.58533357, 1.07678807), (1.70341165e-13, -5.68835427e-14, -9.30120236e-15))]>

<br><br>

- - - 

<span style='font-size:40px;font-weight:650'>
    END
</span>